In [26]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
def open_input(day='08', suffix="input"):
    with open(f"{day}-{suffix}", "r") as file:
        lines = file.readlines()
    data_raw = [line.replace("\n", "") for line in lines]
    data_raw = "\n".join(data_raw)
    return data_raw

input_raw = open_input()
input_raw

'96616,810,75978\n47,21512,72240\n37204,32405,96978\n29869,39703,17245\n85469,86947,73441\n30718,17071,51096\n41362,30138,67009\n598,13948,14379\n76389,15190,48555\n49893,73440,15307\n29849,31150,65844\n43789,38501,72225\n1289,51970,81957\n94777,88446,92003\n67115,73782,47139\n34918,73782,76791\n10226,21601,9106\n25255,34515,55484\n15907,71527,29965\n81513,47000,15653\n80059,88130,47907\n38982,78390,65861\n52404,53380,32169\n57890,49601,15440\n124,21347,33712\n34031,15726,61115\n74713,70099,55008\n5651,21087,42325\n7646,22850,56910\n77521,99356,56389\n21748,86110,49763\n33638,70698,70272\n72192,16208,6392\n44081,77872,46151\n77812,33943,16761\n73761,81089,45679\n8774,27775,53496\n7575,13375,22609\n58036,91575,80779\n59110,66473,56545\n86077,10459,63254\n66197,87350,40327\n81411,92072,72676\n49748,9698,89965\n78439,17433,63349\n21784,15350,87174\n20127,8208,29553\n68687,89811,29134\n37359,84624,5746\n16185,94710,97793\n18231,95999,20306\n46807,93766,75186\n66175,64946,13827\n2379,85923,

In [4]:
input_test_raw = open_input( suffix="test")
input_test_raw

'162,817,812\n57,618,57\n906,360,560\n592,479,940\n352,342,300\n466,668,158\n542,29,236\n431,825,988\n739,650,466\n52,470,668\n216,146,977\n819,987,18\n117,168,530\n805,96,715\n346,949,466\n970,615,88\n941,993,340\n862,61,35\n984,92,344\n425,690,689'

In [6]:
def preprocess_data (data):
    # dtype='U10'
    ranges = []
    for row in data.split("\n"):   
        ranges.append([int(a) for a in row.split(",")])
    
    return ranges
test_data = preprocess_data(input_test_raw)
input_data = preprocess_data(input_raw)
display(test_data)

[[162, 817, 812],
 [57, 618, 57],
 [906, 360, 560],
 [592, 479, 940],
 [352, 342, 300],
 [466, 668, 158],
 [542, 29, 236],
 [431, 825, 988],
 [739, 650, 466],
 [52, 470, 668],
 [216, 146, 977],
 [819, 987, 18],
 [117, 168, 530],
 [805, 96, 715],
 [346, 949, 466],
 [970, 615, 88],
 [941, 993, 340],
 [862, 61, 35],
 [984, 92, 344],
 [425, 690, 689]]

In [24]:
def solution(data, n_connections):
    solution = 0
    
    def euclidian_distance(a, b):
        return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2)
    
    box_mapping = {i: data[i] for i in range(len(data))}
    
    distances = {}
    ordered_distances = []
    
    for comb in combinations(range(len(data)), 2):
        a = data[comb[0]]
        b = data[comb[1]]
        dist = euclidian_distance(a, b)
        
        if not ordered_distances:
            ordered_distances.append((comb, dist))
        else:    
            idx = -1
            for i, d in enumerate(ordered_distances):
                if dist < d[1]:
                    idx = i
                    break
                
            if idx == -1:
                ordered_distances.append((comb, dist))
            elif idx == 0:
                ordered_distances = [(comb, dist)] + ordered_distances
            else:
                ordered_distances = ordered_distances[:idx] + [(comb, dist)] + ordered_distances[idx:]
        
            ordered_distances = ordered_distances[:n_connections]
        
    old_circuits = []
    for pair, dist in ordered_distances[:n_connections]:
        new_circuits = []
        new_circuit = set([pair[0], pair[1]])
        for circuit in old_circuits:
            if pair[0] in circuit or pair[1] in circuit:
                new_circuit = new_circuit.union(circuit)
            else:
                new_circuits.append(circuit)
        
        new_circuits.append(new_circuit) 
        old_circuits = new_circuits
        
    sizes = sorted([len(circuit) for circuit in new_circuits], reverse=True)
    return math.prod(sizes[:3])

sol_test = solution(test_data, 10)
print("Test Solution:", sol_test)
sol_input = solution(input_data, 1000)
print("Input Solution:", sol_input)

Test Solution: 40
Input Solution: 127551


In [37]:


def solution2(data):
    solution = 1
    
    
    def euclidian_distance(a, b):
        return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2)
    
    box_mapping = {i: data[i] for i in range(len(data))}
    
    distances = []
    ordered_distances = []
    
    for comb in tqdm(combinations(range(len(data)), 2)):
        a = data[comb[0]]
        b = data[comb[1]]
        dist = euclidian_distance(a, b)
        distances.append((comb, dist))
        
    distances.sort(key=lambda x: x[1])
               
        
    old_circuits = []
    for pair, dist in distances:
        new_circuits = []
        new_circuit = set([pair[0], pair[1]])
        for circuit in old_circuits:
            if pair[0] in circuit or pair[1] in circuit:
                new_circuit = new_circuit.union(circuit)
            else:
                new_circuits.append(circuit)
        
        new_circuits.append(new_circuit) 
        old_circuits = new_circuits
        if len(new_circuits) == 1 and len(new_circuits[0]) == len(data):
            break
        
    solution = data[pair[0]][0] * data[pair[1]][0]
    return solution


sol_test = solution2(test_data)
print("Test Solution:", sol_test)
sol_input = solution2(input_data)
print("Input Solution:", sol_input)

190it [00:00, 735842.81it/s]


Test Solution: 25272


499500it [00:00, 1110462.91it/s]


Input Solution: 2347225200
